In [ ]:
!pip -q install -U "transformers>=4.52" "datasets>=2.20" "evaluate>=0.4.2" "accelerate>=1.0" \
                 rouge-score nltk absl-py

import nltk, torch, transformers, datasets, evaluate
nltk.download("punkt", quiet=True)
try: nltk.download("punkt_tab", quiet=True)
except: pass

print("Torch:", torch.__version__, "| CUDA:", torch.cuda.is_available())
print("Transformers:", transformers.__version__)


Torch: 2.6.0+cu124 | CUDA: True
Transformers: 4.55.2


In [ ]:
from google.colab import files
print("Upload your MIMIC-III CSV (e.g., mimic3_radiology_pairs.csv)")
up = files.upload()
CSV_PATH = list(up.keys())[0]
CSV_PATH


Upload your MIMIC-III CSV (e.g., mimic3_radiology_pairs.csv)


Saving mimic3_radiology_pairs.csv to mimic3_radiology_pairs (4).csv


'mimic3_radiology_pairs (4).csv'

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

SEED = 42
FAST_DEBUG = False  # True = quick subset run

df = pd.read_csv(CSV_PATH).dropna(subset=["input_text","target_summary"])
df = df.drop_duplicates(subset=["input_text"]).reset_index(drop=True)

# Optional quality filter:
# df = df[(df.input_text.str.len() >= 300) & (df.input_text.str.split().str.len() >= 50)]

if FAST_DEBUG:
    df = df.sample(n=min(8000, len(df)), random_state=SEED).reset_index(drop=True)

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=SEED)
val_df,  test_df  = train_test_split(temp_df, test_size=0.5, random_state=SEED)

dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df,  preserve_index=False),
    "validation": Dataset.from_pandas(val_df, preserve_index=False),
    "test": Dataset.from_pandas(test_df,    preserve_index=False),
})
len(train_df), len(val_df), len(test_df)


(20035, 2504, 2505)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

BIOBART_ID = "GanjinZero/biobart-v2-base"   # try -large later if your GPU allows
MAX_INPUT_LEN  = 512
MAX_TARGET_LEN = 128
OUTPUT_DIR     = "/content/biobart_mimic3"  # keep different from your CXR folder

tokenizer = AutoTokenizer.from_pretrained(BIOBART_ID, use_fast=True)
model     = AutoModelForSeq2SeqLM.from_pretrained(BIOBART_ID)

print("Loaded model:", getattr(model.config, "_name_or_path", None), "| type:", model.config.model_type)


Loaded model: GanjinZero/biobart-v2-base | type: bart


In [ ]:
def preprocess(batch):
    enc = tokenizer(batch["input_text"], max_length=MAX_INPUT_LEN,
                    truncation=True, padding="max_length")
    lab = tokenizer(text_target=batch["target_summary"], max_length=MAX_TARGET_LEN,
                    truncation=True, padding="max_length")
    enc["labels"] = lab["input_ids"]
    return enc

tokenized = dataset.map(preprocess, batched=True, remove_columns=dataset["train"].column_names)
tokenized


Map:   0%|          | 0/20035 [00:00<?, ? examples/s]

Map:   0%|          | 0/2504 [00:00<?, ? examples/s]

Map:   0%|          | 0/2505 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 20035
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2504
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2505
    })
})

In [ ]:
import numpy as np, torch, evaluate
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

rouge = evaluate.load("rouge")
collator = DataCollatorForSeq2Seq(tokenizer, model=model)

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    if isinstance(preds, tuple): preds = preds[0]
    decoded_preds  = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return {k: round(v,4) for k,v in rouge.compute(predictions=decoded_preds,
                                                   references=decoded_labels,
                                                   use_aggregator=True).items()}

base_kwargs = dict(
    output_dir=OUTPUT_DIR,
    learning_rate=2e-5,
    per_device_train_batch_size=4,   # if OOM → 2 or 1
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,   # effective batch 8
    num_train_epochs=1,              # start with 1; later 2–3
    logging_steps=100,
    report_to="none",
)

modern_kwargs = dict(
    evaluation_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=2,
    predict_with_generate=True,
    generation_max_length=MAX_TARGET_LEN,
    fp16=torch.cuda.is_available(),
    load_best_model_at_end=True,
    metric_for_best_model="rougeL",
    greater_is_better=True,
)

use_eval_during_training = True
try:
    args = Seq2SeqTrainingArguments(**base_kwargs, **modern_kwargs)
except TypeError:
    # Old transformers: no evaluation_strategy
    use_eval_during_training = False
    args = Seq2SeqTrainingArguments(
        **base_kwargs,
        save_strategy="no",
        fp16=torch.cuda.is_available(),
    )

trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=(tokenized["validation"] if use_eval_during_training else None),
    tokenizer=tokenizer,
    data_collator=collator,
    compute_metrics=(compute_metrics if use_eval_during_training else None),
)

trainer.train()


/tmp/ipython-input-250526864.py:54: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss
100,2.067900
200,1.517000
300,1.361200
400,1.256900
500,1.257300
600,1.225400
700,1.254700
800,1.150500
900,1.191600
1000,1.124700


TrainOutput(global_step=2505, training_loss=1.187082041285471, metrics={'train_runtime': 316.2379, 'train_samples_per_second': 63.354, 'train_steps_per_second': 7.921, 'total_flos': 6108034970419200.0, 'train_loss': 1.187082041285471, 'epoch': 1.0})

In [ ]:
if use_eval_during_training:
    test_metrics = trainer.evaluate(tokenized["test"])
else:
    # Manual evaluation (older transformers)
    model.eval()
    preds, refs = [], []
    step = 16
    subset = tokenized["validation"].select(range(min(512, tokenized["validation"].num_rows)))
    for i in range(0, len(subset), step):
        b = subset[i:i+step]
        with torch.no_grad():
            gen = model.generate(
                input_ids=torch.tensor(b["input_ids"]).to(model.device),
                attention_mask=torch.tensor(b["attention_mask"]).to(model.device),
                max_new_tokens=MAX_TARGET_LEN, num_beams=4)
        preds.extend(tokenizer.batch_decode(gen, skip_special_tokens=True))
        lab = np.array(b["labels"])
        lab = np.where(lab != -100, lab, tokenizer.pad_token_id)
        refs.extend(tokenizer.batch_decode(lab, skip_special_tokens=True))
    test_metrics = rouge.compute(predictions=preds, references=refs, use_aggregator=True)
    test_metrics = {k: round(v,4) for k,v in test_metrics.items()}

print("ROUGE:", test_metrics)

trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)


ROUGE: {'rouge1': np.float64(0.3709), 'rouge2': np.float64(0.1798), 'rougeL': np.float64(0.2848), 'rougeLsum': np.float64(0.3553)}


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3917: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


('/content/biobart_mimic3/tokenizer_config.json',
 '/content/biobart_mimic3/special_tokens_map.json',
 '/content/biobart_mimic3/vocab.json',
 '/content/biobart_mimic3/merges.txt',
 '/content/biobart_mimic3/added_tokens.json',
 '/content/biobart_mimic3/tokenizer.json')

In [ ]:
import torch
def summarize_findings(text, max_new_tokens=128, num_beams=4):
    ins = tokenizer([text], return_tensors="pt", truncation=True, max_length=MAX_INPUT_LEN).to(model.device)
    with torch.no_grad():
        gen = model.generate(**ins, max_new_tokens=max_new_tokens, num_beams=num_beams,
                             no_repeat_ngram_size=3)
    return tokenizer.decode(gen[0], skip_special_tokens=True)

sample = dataset["test"][0]
print("FINDINGS:\n", sample["input_text"][:800], "\n")
print("REFERENCE:\n", sample["target_summary"], "\n")
print("BIOBART:\n", summarize_findings(sample["input_text"]))


FINDINGS:
 There is no evidence of large pulmonary embolism in the central pulmonary
 arterial vasculature. However, small subsegmental pulmonary embolism cannot
 be excluded. There is no pneumothorax or pleural effusion. There is minimal
 bilateral dependent atelectasis. The lung parenchyma is otherwise clear.

 In the anterior mediastinal space, there is a large predominantly fat- density
 mass but with punctated calcification, compatible with a teratoma. The mass
 measures 10 x 8 x 12 cm (transverse x AP x SI). Upon review from series of
 prior exams, this mass has enlarged in size form prior exams dating back to
 .

 The heart appeared to be slightly laterally displaced to the left. There are
 significant coronary calcifications but no pericardial effusion. The aorta is
 normal in caliber through 

REFERENCE:
 1. No aortic dissection or aneurysm.

 2. Suboptimal IV bolus timing, but no large pulmonary embolism in the central
 vasculature.

 3. Large anterior mediastinal mass with p

In [ ]:
import pandas as pd
import torch

# Pick split: test set (gold-standard evaluation)
test_ds = dataset["test"]

preds, refs = [], []
step = 16  # batch size for generation
for i in range(0, len(test_ds), step):
    batch = test_ds[i:i+step]
    inputs = tokenizer(batch["input_text"], return_tensors="pt",
                       padding=True, truncation=True,
                       max_length=MAX_INPUT_LEN).to(model.device)
    with torch.no_grad():
        gen_ids = model.generate(**inputs, max_new_tokens=MAX_TARGET_LEN,
                                 num_beams=4, no_repeat_ngram_size=3)
    preds.extend(tokenizer.batch_decode(gen_ids, skip_special_tokens=True))
    refs.extend(batch["target_summary"])

# Build DataFrame
out_df = pd.DataFrame({
    "input_text": [ex for ex in test_ds["input_text"]],
    "reference_summary": refs,
    "predicted_summary": preds,
})

# Save to CSV
CSV_OUT = "/content/mimic3_biobart_predictions.csv"
out_df.to_csv(CSV_OUT, index=False)
print("✅ Saved predictions to:", CSV_OUT)


✅ Saved predictions to: /content/mimic3_biobart_predictions.csv
